# Rag From Scratch: Query Transformations

Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

![Screenshot 2024-03-25 at 8.08.30 PM.png](attachment:d9d5305c-e5bb-4934-b91d-5988c87fd767.png)

## Set Environment Vars and API Keys

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQQ_API_KEY")

## Part 5: Multi Query

Flow:

![Screenshot 2024-02-12 at 12.39.59 PM.png](attachment:9efe017a-075f-4017-abef-174c755b11c6.png)

Docs:

* https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever

### Index

In [ ]:
#### INDEXING ####

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = FAISS.from_documents(documents=splits,
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

c:\Vasanth\anaconda\envs\markdown-validation-crew\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Multi Query Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatGroq(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
generate_queries.invoke("What is Task Decomposition")

['1. Can you explain the concept of task decomposition and its significance?',
 '2. How does task decomposition work in the context of problem-solving?',
 '3. What are the steps involved in breaking down a complex task through task decomposition?',
 '4. Can you provide real-world examples that illustrate the use of task decomposition?',
 '5. What is the role of task decomposition in optimizing workflow efficiency?']

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

c:\Vasanth\anaconda\envs\markdown-validation-crew\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


6

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down a complex task into smaller and more manageable sub-tasks. This can be achieved through various methods, such as using simple prompting techniques like "Steps for XYZ" or "What are the subgoals for achieving XYZ?", employing task-specific instructions, or incorporating human inputs.\n\nOne popular prompting technique is Chain of Thought (CoT), which instructs the model to "think step by step" and decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and sheds light on the model\'s thinking process.\n\nAn extension of CoT is Tree of Thoughts, which explores multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by

## Part 6: RAG-Fusion

Flow:

![Screenshot 2024-02-12 at 12.41.36 PM.png](attachment:0bc49f5b-c338-4cd4-ac04-8744994e0e81.png)


### Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

generate_queries = (
    prompt_rag_fusion
    | ChatGroq(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

## RRF Explanation

#### Question Rankings

- **Question A**:
  1. Doc1
  2. Doc4
  3. Doc3
  4. Doc2

- **Question B**:
  1. Doc3
  2. Doc1
  3. Doc2
  4. Doc4

- **Question C**:
  1. Doc4
  2. Doc3
  3. Doc1
  4. Doc2

### Rank Positions

- **Doc1**:
  - Question A rank: 1
  - Question B rank: 2
  - Question C rank: 3

- **Doc2**:
  - Question A rank: 4
  - Question B rank: 3
  - Question C rank: 4

- **Doc3**:
  - Question A rank: 3
  - Question B rank: 1
  - Question C rank: 2

- **Doc4**:
  - Question A rank: 2
  - Question B rank: 4
  - Question C rank: 1

### Reciprocal Rank Fusion Calculation

Using `k = 60`:

#### Doc1
- Reciprocal Rank (Question A): `1 / (60 + 1) = 1 / 61`
- Reciprocal Rank (Question B): `1 / (60 + 2) = 1 / 62`
- Reciprocal Rank (Question C): `1 / (60 + 3) = 1 / 63`
- **RRF(Doc1)**: `1 / 61 + 1 / 62 + 1 / 63 ≈ 0.0487`

#### Doc2
- Reciprocal Rank (Question A): `1 / (60 + 4) = 1 / 64`
- Reciprocal Rank (Question B): `1 / (60 + 3) = 1 / 63`
- Reciprocal Rank (Question C): `1 / (60 + 4) = 1 / 64`
- **RRF(Doc2)**: `1 / 64 + 1 / 63 + 1 / 64 ≈ 0.0469`

#### Doc3
- Reciprocal Rank (Question A): `1 / (60 + 3) = 1 / 63`
- Reciprocal Rank (Question B): `1 / (60 + 1) = 1 / 61`
- Reciprocal Rank (Question C): `1 / (60 + 2) = 1 / 62`
- **RRF(Doc3)**: `1 / 63 + 1 / 61 + 1 / 62 ≈ 0.0487`

#### Doc4
- Reciprocal Rank (Question A): `1 / (60 + 2) = 1 / 62`
- Reciprocal Rank (Question B): `1 / (60 + 4) = 1 / 64`
- Reciprocal Rank (Question C): `1 / (60 + 1) = 1 / 61`
- **RRF(Doc4)**: `1 / 62 + 1 / 64 + 1 / 61 ≈ 0.0484`

### Final Ranking

Based on the RRF scores:

1. **Doc1**: `≈ 0.0487`
2. **Doc3**: `≈ 0.0487`
3. **Doc4**: `≈ 0.0484`
4. **Doc2**: `≈ 0.0469`

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

7

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down large tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done using simple prompting techniques, task-specific instructions, or with human inputs. LLMs like HuggingGPT use task planning and parsing to decompose user requests into multiple tasks with associated attributes. However, there are challenges in long-term planning and task decomposition, such as the limited context length and the reliability of the natural language interface.'

## Part 7: Decomposition

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [ ]:
questions

['1. "Main components of a language model in autonomous agent systems"',
 '2. "Role of natural language understanding in LLM-powered autonomous agents"',
 '3. "How does natural language generation contribute to autonomous agent systems with LLMs?"']

### Answer recursively  

![Screenshot 2024-02-18 at 1.55.32 PM.png](attachment:9a9685de-051f-48fa-b68f-2b1f85344cdf.png)

In [ ]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatGroq(temperature=0)

q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [ ]:
answer

'Natural language generation (NLG) contributes to autonomous agent systems with LLMs (large language models) by enabling the agent to process, comprehend, and generate human-like language. This is crucial for the agent to communicate effectively with users and external components, as well as to handle complex tasks through subgoal decomposition and self-reflection.\n\nThe LLM at the core of the autonomous agent system is responsible for natural language generation, and it can perform various NLG tasks such as text generation, translation, summarization, and question-answering. The LLM can generate natural language outputs based on the given goals and past actions, and it can learn from its mistakes through self-reflection and self-criticism.\n\nMoreover, the LLM can generate natural language inputs for external components such as memory and tools, and it can process natural language inputs from the user and external components to make decisions and take actions. The natural language in

### Answer individually

![Screenshot 2024-02-18 at 2.00.59 PM.png](attachment:e24502d7-f641-4262-a326-da1636822fa2.png)

In [ ]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        rag_results.append(answer)

    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

c:\Vasanth\anaconda\envs\markdown-validation-crew\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"The main components of an LLM-powered autonomous agent system include the language model (LLM) and the system message. The LLM, which can make decisions independently, serves as the brain for natural language understanding, enabling complex task handling through subgoal decomposition, learning from past actions through reflection and refinement, and maintaining memory. Meanwhile, the system message defines the AI's name, description, and goals. Additionally, natural language generation acts as the main controller, allowing the system to handle complex tasks, decompose subgoals, and provide self-reflection for continuous learning and improvement. However, there are common challenges across these components, such as limited context length, difficulties in long-term planning and task decomposition, and reliability issues with the natural language interface."

## Part 8: Step Back

![Screenshot 2024-02-12 at 1.14.43 PM.png](attachment:715e11dc-7730-4f51-8469-b7f0b299ac9e.png)

In [ ]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [ ]:
generate_queries_step_back = prompt | ChatGroq(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

"How is a large language model agent's task broken down? \n\nor\n\nWhat are the steps involved in completing a task for a large language model agent?"

In [ ]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatGroq(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down complex tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently and effectively. There are several methods for task decomposition in LLM agents, including:\n\n1. Simple prompting: The LLM is prompted with simple instructions, such as "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?" to decompose a task.\n2. Task-specific instructions: The LLM is provided with specific instructions for a particular task, such as "Write a story outline." for writing a novel.\n3. Human inputs: The LLM can also receive task decomposition input from human users.\n\nAdditionally, there are advanced techniques such as Chain of Thought (CoT) and Tree of Thoughts (ToT) that enhance model performance on complex tasks by decomposing hard tasks into smaller and simpler steps, and exploring multiple reasoning possibilities at each step, respectively.\n\

## Part 9: HyDE

![Screenshot 2024-02-12 at 1.12.45 PM.png](attachment:1982149e-720b-426e-a1ab-8d96f6616b5a.png)

In [ ]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | ChatGroq(temperature=0) | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition is a crucial aspect of building Large Language Model (LLM) agents that can effectively perform complex tasks. It involves breaking down a complex task into smaller, manageable subtasks that the LLM agent can handle more efficiently. This process is similar to the way humans approach complex problems by dividing them into smaller, more manageable parts.\n\nIn the context of LLM agents, task decomposition enables the agent to focus on one subtask at a time, reducing the cognitive load and increasing the chances of success. Each subtask is defined in a way that the LLM agent can understand and execute, often involving natural language instructions.\n\nFor example, consider a task where an LLM agent is required to write a scientific paper. The agent can decompose this task into several subtasks, such as:\n\n1. Understanding the research question\n2. Conducting a literature review\n3. Developing a thesis statement\n4. Outlining the paper\n5. Writing the introduction\n6. 

In [ ]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via 

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down large tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done using simple prompting techniques, task-specific instructions, or with human inputs. The document also mentions a technique called Tree of Thoughts, which extends the concept of task decomposition by exploring multiple reasoning possibilities at each step, creating a tree structure. The search process in Tree of Thoughts can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier or majority vote.'